<a href="https://colab.research.google.com/github/omrani-wiem/stage_code_alpha/blob/main/stage_code_alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Télécharge le fichier kaggle.json que tu as récupéré
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"omraniwiem","key":"d8aaf2feb2f6fa41c0e0e266bb37c9b8"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Installer l'API Kaggle
!pip install kaggle

In [6]:
!kaggle datasets download -d mdaliraza/german-credit-data


Dataset URL: https://www.kaggle.com/datasets/mdaliraza/german-credit-data
License(s): CC0-1.0
german-credit-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!ls

german-credit-data.zip	kaggle.json  sample_data


In [8]:
!unzip -o german-credit-data.zip -d ./data

Archive:  german-credit-data.zip
  inflating: ./data/german_credit_data.csv  


In [9]:
!ls ./data

german_credit_data.csv


In [20]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [10]:
import pandas as pd

df = pd.read_csv('./data/german_credit_data.csv')
df.head()


,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,0,67,male,2,own,NaN,little,1169,6,radio/TV
1,1,22,female,2,own,little,moderate,5951,48,radio/TV
2,2,49,male,1,own,little,NaN,2096,12,education
3,3,45,male,2,free,little,little,7882,42,furniture/equipment
4,4,53,male,2,free,little,little,4870,24,car


In [11]:
df = df.drop(columns=['Unnamed: 0'] , errors='ignore')

In [16]:
df.isnull().sum()

,0
Age,0
Sex,0
Job,0
Housing,0
Saving accounts,183
Checking account,394
Credit amount,0
Duration,0
Purpose,0


In [18]:
df['Saving accounts'] = df['Saving accounts'].fillna('none')
df['Checking account'] = df['Checking account'].fillna('none')

In [32]:
print(df.dtypes)

AttributeError: 'numpy.ndarray' object has no attribute 'dtypes'